## Over deze opdrachten

* dit is Jupyter Notebook `python-sqlite-0.ipynb` - voor het aanmaken van de database.
* voor een inleiding over het gebruik van Jupyter Notebooks: [Inleiding Jupyter Notebook](Inleiding-Jupyter.ipynb)
* de hele reeks Python SQlite opdrachten:
    * [Python SQLite - init database](python-sqlite-0.ipynb) (om met een schone lei te beginnnen)
    * [Python SQLite - selectie en projectie](python-sqlite-1.ipynb)    
    * [Python SQLite - joins](python-sqlite-2.ipynb)
    * [Python SQLite - CRUD](python-sqlite-3.ipynb)
    * [Python SQLite - Schema](python-sqlite-4.ipynb)
    
 ### Voorbeeld
 
 Bij deze opdrachten gebruiken we een voorbeeld-database met drie tabellen: `leden`, `inschrijvingen`, en `events`.
 Deze database komt uit een webtoepassing; deze vind je op glitch.com. REF
 Daar kun je de toepassing bekijken, uitproberen, en er een eigen versie ("remix") van maken.

## Aanmaken van de database

In de volgende opdrachten voer je allerlei queries uit op een database.
Je moeten dan eerst wel een database met inhoud hebben.
Met de onderstaande opdrachten maak je deze database.
Deze opdrachten hoef je maar één keer uit te voeren: de database blijft bestaan, met je veranderingen.
Je kunt deze opdrachten ook uitvoeren om opnieuw te beginnen, in een goed gedefinieerde toestand.

In [ ]:
import sqlite3
db = sqlite3.connect('example.db')
cursor = db.cursor()

We maken de tabel(len) aan.
We verwijderen eerst een eventueel bestaande versie van de tabel(len):
we hebben dan een goed gedefinieerde toestand.

> Opmerking: er zijn kleine verschillen in de notatie van de constraints bij het aanmaken van een tabel; MySQL gebruikt bijvoorbeeld een andere notatie dan Oracle.

### Eerste tabel: leden

In [ ]:
cursor.execute('''DROP TABLE IF EXISTS leden;''')
cursor.execute('''CREATE TABLE leden(
                    lidnr INTEGER PRIMARY KEY,
                    voornaam VARCHAR(255) NOT NULL,
                    achternaam VARCHAR(255) NOT NULL,
                    email VARCHAR(255) NOT NULL UNIQUE
                  );''')

We hebben een voorbeeld-inhoud van de tabel(len) in csv-bestanden.
Zo'n csv-bestand kun je gemakkelijk aanpassen in een teksteditor.
Voor het importeren van een csv-bestand gebruiken we een speciale SQLite-opdracht, via de shell.
(Een alternatief is om dit bestand te importeren via pandas.)

In [ ]:
%%bash
sqlite3 example.db
.mode csv
.import leden.csv leden

Hieronder een voorbeeld van een SQL-opdracht die we rechtstreeks in SQLite uitvoeren.

In [ ]:
%%bash
sqlite3 example.db
SELECT * FROM leden;

Eenzelfde opdracht, nu als onderdeel van een Python-programma:

In [ ]:
cursor.execute('''SELECT * FROM leden;''')
for row in cursor:
    print(row)

### Tweede tabel: events

De tabel `events` bevat de events waarvoor de leden kunnen inschrijven.
Elk event heeft een datum en een beschrijving.

Hiervoor volgen we hetzelfde patroon:

In [ ]:
cursor.execute('''DROP TABLE IF EXISTS events;''')
cursor.execute('''CREATE TABLE events(
                  eventnr INTEGER, 
                  datum VARCHAR(255) NOT NULL,
                  beschrijving VARCHAR(255),
                  PRIMARY KEY (eventnr),
                  CONSTRAINT name UNIQUE (datum, beschrijving)
                  );''')

In [ ]:
%%bash
sqlite3 example.db
.mode csv
.import events.csv events

In [ ]:
cursor.execute('''SELECT * FROM events;''')
for row in cursor:
    print(row)

### Derde tabel: inschrijvingen

Deze tabel beschrijft een N-M relatie tussen leden en inschrijvingen.
Naast de verwijzingen (via *foreign keys*) naar de andere tabellen vindt je hier de gegevens over de inschrijving (maaltijd-keuze).

In [ ]:
cursor.execute('''DROP TABLE IF EXISTS inschrijvingen;''')
cursor.execute('''CREATE TABLE inschrijvingen(
                  eventnr INTEGER, 
                  lidnr INTEGER,
                  maaltijd VARCHAR(255),
                  PRIMARY KEY (lidnr, eventnr),
                  FOREIGN KEY (lidnr) REFERENCES leden (lidnr),
                  FOREIGN KEY (eventnr) REFERENCES events (eventnr)
                  );''')

In [ ]:
%%bash
sqlite3 example.db
.mode csv
.import inschrijvingen.csv inschrijvingen

In [ ]:
cursor.execute('''SELECT * FROM inschrijvingen;''')
for row in cursor:
    print(row)

### Demonstratie: alle inschrijvingen

Voor een overzicht van alle inschrijvingen met de gegevens van de leden en van de events gebruiken  we een join. 
Dit is een voorproefje - in een volgend notebook werken we dit verder uit.

In [ ]:
cursor.execute('''SELECT evt.datum, evt.beschrijving, lid.voornaam, lid.achternaam, lid.email, ins.maaltijd
                  FROM inschrijvingen ins, events evt, leden lid 
                  WHERE ins.lidnr = lid.lidnr AND ins.eventnr = evt.eventnr;''')
for row in cursor:
    print(row)

In [ ]:
db.commit()
db.close()